In [8]:
import pandas as pd
import openai
import weaviate
import os
import streamlit as st


df_base = pd.read_excel('Base.xlsx', skiprows = 2, header = 1)
df_base = df_base.iloc[:, :9]

renomear = {"EMPRESA": "EMPRESA",
            "SEGMENTO DE MERCADO": "SEGMENTO_MERCADO",
            "PROCESSO": "PROCESSO",
            "ATIVIDADE RELACIONADA": "ATIVIDADE_RELACIONADA",
            "TIPO DE MELHORIA": "MELHORIA_SUGERIDA",
            "DESCONEXÕES (GAP)": "GAPS",
            "CAUSA": "CAUSA",
            "MELHORIA/SOLUÇÃO": "SOLUCAO",
            "GANHOS/OBJETIVO": "GANHOS"
}

df_base = df_base.rename(columns=renomear)

# Configuração da API OpenAI
import weaviate
from weaviate.auth import AuthApiKey

# Configuração da API OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

# Configuração da conexão com o Weaviate
client = weaviate.Client(
    auth_client_secret=AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY")),  # Autenticação
    url=os.getenv("WEAVIATE_URL"),  # URL do Weaviate
    additional_headers={
        "X-OpenAI-Api-Key": openai.api_key,  # Chave da API OpenAI para vetorização
    }
)

schema = {
    "classes": [
        {
            "class": "DocsOportunidades",
            "description": "Detalhes de melhorias em processos empresariais",
            "properties": [
                {"name": "EMPRESA", "dataType": ["text"], "description": "Nome da empresa"},
                {"name": "SEGMENTO_MERCADO", "dataType": ["text"], "description": "Segmento de mercado"},
                {"name": "PROCESSO", "dataType": ["text"], "description": "Descrição do processo"},
                {"name": "ATIVIDADE_RELACIONADA", "dataType": ["text"], "description": "Atividade relacionada ao processo"},
                {"name": "MELHORIA_SUGERIDA", "dataType": ["text"], "description": "Categoria da melhoria sugerida"},
                {"name": "GAPS", "dataType": ["text"], "description": "Problemas ou lacunas identificadas"},
                {"name": "CAUSA", "dataType": ["text"], "description": "Causa raiz do problema"},
                {"name": "SOLUCAO", "dataType": ["text"], "description": "Proposta de melhoria ou solução"},
                {"name": "GANHOS", "dataType": ["text"], "description": "Ganhos ou objetivos esperados com a solução"}
            ],
            "vectorizer": "text2vec-openai"  # Permite vetorização automática
        }
    ]
}

# Verifica se o schema já existe
if not client.schema.contains({'classes': [{'class': 'DocsOportunidades'}]}):
    client.schema.create(schema)

# Função para gerar embeddings com OpenAI
# Função para gerar embeddings com OpenAI
def generate_embedding(text):
    if not text:  # Garantir que o texto não seja vazio
        return None
    response = openai.Embedding.create(
        model="text-embedding-ada-002",  # Modelo de embeddings
        input=text
    )
    return response['data'][0]['embedding']  # Acessar os dados corretamente


# Iterar sobre as linhas do DataFrame df_base e processar os dados
for _, row in df_base.iterrows():
    # Criar uma string concatenada para gerar embedding
    combined_text = f"{row['MELHORIA_SUGERIDA']} {row['SEGMENTO_MERCADO']} {row['EMPRESA']} {row['PROCESSO']} {row['ATIVIDADE_RELACIONADA']} {row['GAPS']} {row['CAUSA']} {row['SOLUCAO']} {row['GANHOS']}"
    embedding = generate_embedding(combined_text)

    if embedding:  # Verificar se a embedding foi gerada corretamente
        # Adicionar ao Weaviate
        client.data_object.create(
            class_name="DocsOportunidades",
            properties={
                "EMPRESA": row['EMPRESA'],
                "SEGMENTO_MERCADO": row['SEGMENTO_MERCADO'],
                "PROCESSO": row['PROCESSO'],
                "ATIVIDADE_RELACIONADA": row['ATIVIDADE_RELACIONADA'],
                "MELHORIA_SUGERIDA": row['MELHORIA_SUGERIDA'],
                "GAPS": row['GAPS'],
                "CAUSA": row['CAUSA'],
                "SOLUCAO": row['SOLUCAO'],
                "GANHOS": row['GANHOS']
            },
            vector=embedding  # Adiciona o embedding gerado
        )

print("Dados processados e armazenados no Weaviate com sucesso.")

C:\Users\TiagoVettorazzi\AppData\Local\Temp\ipykernel_22860\2278042819.py:32: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(
c:\Users\TiagoVettorazzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\weaviate\warnings.py:186: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 202

KeyError: 'properties'